# Data Science Job Change Prediction
- Author: Le Liu
- Course: COMP3010J Machine Learning


## 1. Project Overview

This project aims to predict whether a candidate is looking for a job change based on various demographic and professional features. Then infer the key factors influencing their decision.

**Dataset:** `data-science-job-change.csv`

**Problem Type:** Binary Classification

**Target Variable:** `target` (1.0 = Looking for job change, 0.0 = Not looking for job change)



*Project Structure*

1. **Introduction** - Project overview and objectives
2. **Load and Analyse Data** - Data loading and initial exploration
3. **Data Cleaning** - Handle missing values and data quality issues
4. **Data Visualisation** - Exploratory data analysis with plots
5. **Attribute Selection** - Feature selection and engineering
6. **Model Selection and Experiments** - Train and compare models
7. **Final Model Training** - Train the best model
8. **Further Analysis and Discussion** - Model interpretation
9. **Discussion** - Conclusions and future work

---
## 2. Data Loading & Analysis

In [264]:
# Load the dataset
import pandas as pd


df = pd.read_csv('data-science-job-change.csv')

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print("\nFirst 5 rows of the dataset:")
df.head()

Dataset shape: (19158, 14)
Number of rows: 19158
Number of columns: 14

First 5 rows of the dataset:


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [265]:
# Display statistical summary
print("Statistical Summary:")
print("="*60)
numerical_cols = ['city_development_index', 'training_hours', 'target']
df[numerical_cols].describe()

Statistical Summary:


,city_development_index,training_hours,target
count,19158.000000,19158.000000,19158.000000
mean,0.828848,65.366896,0.249348
std,0.123362,60.058462,0.432647
min,0.448000,1.000000,0.000000
25%,0.740000,23.000000,0.000000
50%,0.903000,47.000000,0.000000
75%,0.920000,88.000000,0.000000
max,0.949000,336.000000,1.000000


In [266]:
# Check missing values
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(3),
    'Data_Type': df.dtypes,
    'Unique_Values': [df[col].nunique() for col in df.columns]
})

print("Data Health Report:")
print("="*80)
print(missing_data.to_string(index=False))

Data Health Report:
                Column  Missing_Count  Missing_Percentage Data_Type  Unique_Values
           enrollee_id              0               0.000     int64          19158
                  city              0               0.000    object            123
city_development_index              0               0.000   float64             93
                gender           4508              23.531    object              3
   relevent_experience              0               0.000    object              2
   enrolled_university            386               2.015    object              3
       education_level            460               2.401    object              5
      major_discipline           2813              14.683    object              6
            experience             65               0.339    object             22
          company_size           5938              30.995    object              8
          company_type           6140              32.049    object

---

##  3. Data Cleaning & Feature Engineering Strategy

Based on the comprehensive data analysis above, here is the detailed processing strategy for each of the 14 features:

###  Strategy Overview Table

| # | Feature | Type | Action | Priority | Reason |
|---|---------|------|--------|----------|--------|
| 1 | `enrollee_id` | ID | **DELETE** | 🔴 High | No predictive value - just an identifier |
| 2 | `city` | Categorical (123 values) | **Target Encoding** | 🟡 Medium | Too many categories for One-Hot |
| 3 | `city_development_index` | Numerical | **Keep as-is** | 🟢 Low | Already numeric, no missing values |
| 4 | `gender` | Categorical (3 values) | **Fill Missing + One-Hot** | 🟡 Medium | 23% missing, create "Unknown" category |
| 5 | `relevent_experience` | Binary | **Label Encoding** | 🟢 Low | No missing, convert to 0/1 |
| 6 | `enrolled_university` | Categorical (3 values) | **Fill Missing + One-Hot** | 🟢 Low | 2% missing, 3 clear categories |
| 7 | `education_level` | Ordinal (5 values) | **Ordinal Encoding** | 🟡 Medium | 2% missing, has natural order |
| 8 | `major_discipline` | Categorical (6 values) | **Fill Missing + One-Hot** | 🟡 Medium | 15% missing, may correlate with education |
| 9 | `experience` | Ordinal (22 values) | **Ordinal Encoding** | 🔴 High | Convert to numeric years (e.g., "<1"→0, ">20"→21) |
| 10 | `company_size` | Ordinal (8 values) | **Fix Format + Ordinal** | 🔴 High | 31% missing, FIX "10/49" → "10-49" |
| 11 | `company_type` | Categorical (6 values) | **Fill Missing + One-Hot** | 🟡 Medium | 32% missing, create "Unknown" |
| 12 | `last_new_job` | Ordinal (6 values) | **Ordinal Encoding** | 🟢 Low | 2% missing, convert to numeric |
| 13 | `training_hours` | Numerical | **Keep as-is** | 🟢 Low | Already numeric, no missing values |
| 14 | `target` | Binary | **Keep as-is** | N/A | Target variable - no transformation needed |

---

### Data Cleaning Implementation Plan


In [267]:
# Create a copy of the original dataframe for cleaning
df_clean = df.copy()

print(f"Original dataset shape: {df_clean.shape}")
print(f"Starting data cleaning process...\n")

Original dataset shape: (19158, 14)
Starting data cleaning process...



In [268]:
# Feature 1: enrollee_id
df_clean = df_clean.drop('enrollee_id', axis=1)

print("Feature 1 processed: enrollee_id")
print(f"Action: Deleted")
print(f"New shape: {df_clean.shape}")
print(f"Columns remaining: {df_clean.shape[1]}\n")

Feature 1 processed: enrollee_id
Action: Deleted
New shape: (19158, 13)
Columns remaining: 13



In [269]:
# Feature 2: company_size
print("="*70)
print("Processing Feature 2: company_size")
print("="*70)

# Step 1: Fix formatting error
print("\nStep 1: Fix formatting error '10/49' → '10-49'")
before_fix = df_clean['company_size'].value_counts(dropna=False)
df_clean['company_size'] = df_clean['company_size'].replace('10/49', '10-49')
after_fix = df_clean['company_size'].value_counts(dropna=False)
print(f"Fixed: '10/49' no longer present")

# Step 2: Handle missing values
print("\nStep 2: Fill missing values with 'Unknown'")
missing_count = df_clean['company_size'].isna().sum()
print(f"   Missing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")
df_clean['company_size'] = df_clean['company_size'].fillna('Unknown')
print(f"   Missing after: {df_clean['company_size'].isna().sum()}")

# Step 3: Ordinal encoding
print("\nStep 3: Ordinal encoding (small → large)")
size_order = {
    '<10': 0,          # 最小
    '10-49': 1,
    '50-99': 2,
    '100-500': 3,
    '500-999': 4,
    '1000-4999': 5,
    '5000-9999': 6,
    '10000+': 7,       # 最大
    'Unknown': -1      # ← 特殊值,不在顺序中
}
df_clean['company_size'] = df_clean['company_size'].map(size_order)

print("   Encoding mapping:")
for key, value in size_order.items():
    count = (df_clean['company_size'] == value).sum()
    print(f"      {key:12s} → {value}  ({count:,} samples)")

print(f"\n Feature 2 processed: company_size")
print(f"   Type: {df_clean['company_size'].dtype}")
print(f"   Range: {df_clean['company_size'].min()} to {df_clean['company_size'].max()}")
print(f"   Missing: {df_clean['company_size'].isna().sum()}\n")

Processing Feature 2: company_size

Step 1: Fix formatting error '10/49' → '10-49'
Fixed: '10/49' no longer present

Step 2: Fill missing values with 'Unknown'
   Missing before: 5938 (30.99%)
   Missing after: 0

Step 3: Ordinal encoding (small → large)
   Encoding mapping:
      <10          → 0  (1,308 samples)
      10-49        → 1  (1,471 samples)
      50-99        → 2  (3,083 samples)
      100-500      → 3  (2,571 samples)
      500-999      → 4  (877 samples)
      1000-4999    → 5  (1,328 samples)
      5000-9999    → 6  (563 samples)
      10000+       → 7  (2,019 samples)
      Unknown      → -1  (5,938 samples)

 Feature 2 processed: company_size
   Type: int64
   Range: -1 to 7
   Missing: 0



In [270]:
# Feature 3: company_type
print("="*70)
print("Processing Feature 3: company_type")
print("="*70)

# Step 1: Handle missing values
print("\nStep 1: Fill missing values with 'Unknown'")
missing_count = df_clean['company_type'].isna().sum()
print(f"Missing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")
df_clean['company_type'] = df_clean['company_type'].fillna('Unknown')
print(f"Missing after: {df_clean['company_type'].isna().sum()}")

# Step 2: One-Hot encoding
print("\nStep 2: One-Hot encoding")
print(f"Categories: {sorted(df_clean['company_type'].unique())}")

company_type_dummies = pd.get_dummies(df_clean['company_type'], prefix='company_type', drop_first=False)
df_clean = pd.concat([df_clean, company_type_dummies], axis=1)
df_clean = df_clean.drop('company_type', axis=1)

print(f"Created {len(company_type_dummies.columns)} binary columns:")
for col in sorted(company_type_dummies.columns):
    count = df_clean[col].sum()
    print(f"      {col:30s}: {count:,} samples")

print(f"\nFeature 3 processed: company_type")
print(f"   New columns added: {len(company_type_dummies.columns)}")
print(f"   Current shape: {df_clean.shape}\n")

Processing Feature 3: company_type

Step 1: Fill missing values with 'Unknown'
Missing before: 6140 (32.05%)
Missing after: 0

Step 2: One-Hot encoding
Categories: ['Early Stage Startup', 'Funded Startup', 'NGO', 'Other', 'Public Sector', 'Pvt Ltd', 'Unknown']
Created 7 binary columns:
      company_type_Early Stage Startup: 603 samples
      company_type_Funded Startup   : 1,001 samples
      company_type_NGO              : 521 samples
      company_type_Other            : 121 samples
      company_type_Public Sector    : 955 samples
      company_type_Pvt Ltd          : 9,817 samples
      company_type_Unknown          : 6,140 samples

Feature 3 processed: company_type
   New columns added: 7
   Current shape: (19158, 19)



In [271]:
# Feature 4: gender
print("="*70)
print("Processing Feature 4: gender")
print("="*70)

missing_count = df_clean['gender'].isna().sum()
print(f"Missing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")
df_clean['gender'] = df_clean['gender'].fillna('Unknown')

gender_dummies = pd.get_dummies(df_clean['gender'], prefix='gender', drop_first=False)
df_clean = pd.concat([df_clean, gender_dummies], axis=1)
df_clean = df_clean.drop('gender', axis=1)

print(f"Created {len(gender_dummies.columns)} columns: {list(gender_dummies.columns)}")
print(f"Current shape: {df_clean.shape}\n")

Processing Feature 4: gender
Missing before: 4508 (23.53%)
Created 4 columns: ['gender_Female', 'gender_Male', 'gender_Other', 'gender_Unknown']
Current shape: (19158, 22)

Created 4 columns: ['gender_Female', 'gender_Male', 'gender_Other', 'gender_Unknown']
Current shape: (19158, 22)



In [272]:
# Feature 5: major_discipline
print("="*70)
print("Processing Feature 5: major_discipline")
print("="*70)

missing_count = df_clean['major_discipline'].isna().sum()
print(f"Missing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")

# Fill missing with 'Unknown'
print("\nFilling strategy: All missing → 'Unknown'")
df_clean['major_discipline'] = df_clean['major_discipline'].fillna('Unknown')
print(f"Filled {missing_count} with 'Unknown'")
print(f"Missing after: {df_clean['major_discipline'].isna().sum()}")

# One-Hot encoding
major_dummies = pd.get_dummies(df_clean['major_discipline'], prefix='major', drop_first=False)
df_clean = pd.concat([df_clean, major_dummies], axis=1)
df_clean = df_clean.drop('major_discipline', axis=1)

print(f"\nCreated {len(major_dummies.columns)} columns: {list(major_dummies.columns)}")
print(f"Current shape: {df_clean.shape}\n")

Processing Feature 5: major_discipline
Missing before: 2813 (14.68%)

Filling strategy: All missing → 'Unknown'
Filled 2813 with 'Unknown'
Missing after: 0

Created 7 columns: ['major_Arts', 'major_Business Degree', 'major_Humanities', 'major_No Major', 'major_Other', 'major_STEM', 'major_Unknown']
Current shape: (19158, 28)



In [273]:
# Feature 6: experience
print("="*70)
print("Processing Feature 6: experience")
print("="*70)

missing_count = df_clean['experience'].isna().sum()
print(f"Missing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")

# First convert to numeric for median calculation
exp_mapping = {
    '<1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5,
    '6': 6, '7': 7, '8': 8, '9': 9, '10': 10,
    '11': 11, '12': 12, '13': 13, '14': 14, '15': 15,
    '16': 16, '17': 17, '18': 18, '19': 19, '20': 20,
    '>20': 25 
}

# Fill missing with median before encoding
# Calculate median from non-missing values
temp_numeric = df_clean['experience'].dropna().map(exp_mapping)
median_value = temp_numeric.median()
# Find the closest original value to median
median_key = min(exp_mapping.items(), key=lambda x: abs(x[1] - median_value))[0]
df_clean['experience'] = df_clean['experience'].fillna(median_key)
print(f"   Filled {missing_count} missing with median experience: '{median_key}' (value={exp_mapping[median_key]})")

# Apply ordinal encoding
df_clean['experience'] = df_clean['experience'].map(exp_mapping)

print(f"\nFeature 6 processed: experience")
print(f"   Type: {df_clean['experience'].dtype}")
print(f"   Range: {df_clean['experience'].min()} to {df_clean['experience'].max()} years")
print(f"   Missing: {df_clean['experience'].isna().sum()}\n")

Processing Feature 6: experience
Missing before: 65 (0.34%)
   Filled 65 missing with median experience: '9' (value=9)

Feature 6 processed: experience
   Type: int64
   Range: 0 to 25 years
   Missing: 0


Processing Feature 6: experience
Missing before: 65 (0.34%)
   Filled 65 missing with median experience: '9' (value=9)

Feature 6 processed: experience
   Type: int64
   Range: 0 to 25 years
   Missing: 0



In [274]:
# Feature 7: education_level
print("="*70)
print("Processing Feature 7: education_level")
print("="*70)

missing_count = df_clean['education_level'].isna().sum()
mode_value = df_clean['education_level'].mode()[0]
df_clean['education_level'] = df_clean['education_level'].fillna(mode_value)
print(f"Filled {missing_count} missing with mode: '{mode_value}'")

# Ordinal encoding: Lower education -> Higher values
edu_mapping = {
    'Primary School': 1,
    'High School': 2,
    'Graduate': 3,
    'Masters': 4,
    'Phd': 5
}
df_clean['education_level'] = df_clean['education_level'].map(edu_mapping)

print(f"\nFeature 7 processed: education_level")
print(f"Ordinal encoded: range {df_clean['education_level'].min()}-{df_clean['education_level'].max()}")
print(f"Missing: {df_clean['education_level'].isna().sum()}\n")

Processing Feature 7: education_level
Filled 460 missing with mode: 'Graduate'

Feature 7 processed: education_level
Ordinal encoded: range 1-5
Missing: 0

Filled 460 missing with mode: 'Graduate'

Feature 7 processed: education_level
Ordinal encoded: range 1-5
Missing: 0



In [275]:
# Feature 8: enrolled_university
print("="*70)
print("Processing Feature 8: enrolled_university")
print("="*70)

missing_count = df_clean['enrolled_university'].isna().sum()
df_clean['enrolled_university'] = df_clean['enrolled_university'].fillna('no_enrollment')
print(f"Filled {missing_count} missing with 'no_enrollment'")

enrolled_dummies = pd.get_dummies(df_clean['enrolled_university'], prefix='enrolled', drop_first=False)
df_clean = pd.concat([df_clean, enrolled_dummies], axis=1)
df_clean = df_clean.drop('enrolled_university', axis=1)
print(f"Created {len(enrolled_dummies.columns)} columns, shape: {df_clean.shape}\n")

Processing Feature 8: enrolled_university
Filled 386 missing with 'no_enrollment'
Created 3 columns, shape: (19158, 30)



In [276]:
# Feature 9: relevent_experience
print("="*70)
print("Processing Feature 9: relevent_experience")
print("="*70)

print(f"Missing values: {df_clean['relevent_experience'].isna().sum()}")

# Binary encoding: Has relevent experience -> 1, No experience -> 0
df_clean['relevent_experience'] = df_clean['relevent_experience'].map({
    'Has relevent experience': 1,
    'No relevent experience': 0
})

print(f"\n Feature 9 processed: relevent_experience")
print(f"   Binary encoded: {df_clean['relevent_experience'].value_counts().to_dict()}")
print(f"   Missing: {df_clean['relevent_experience'].isna().sum()}\n")

Processing Feature 9: relevent_experience
Missing values: 0

 Feature 9 processed: relevent_experience
   Binary encoded: {1: 13792, 0: 5366}
   Missing: 0



In [277]:
# Feature 10: last_new_job
print("="*70)
print("Processing Feature 10: last_new_job")
print("="*70)

missing_count = df_clean['last_new_job'].isna().sum()
mode_value = df_clean['last_new_job'].mode()[0]
df_clean['last_new_job'] = df_clean['last_new_job'].fillna(mode_value)
print(f"Filled {missing_count} missing with mode: '{mode_value}'")

job_mapping = {'never': 0, '1': 1, '2': 2, '3': 3, '4': 4, '>4': 5}
df_clean['last_new_job'] = df_clean['last_new_job'].map(job_mapping)
print(f"Ordinal encoded: range {df_clean['last_new_job'].min()}-{df_clean['last_new_job'].max()}\n")

Processing Feature 10: last_new_job
Filled 423 missing with mode: '1'
Ordinal encoded: range 0-5

Filled 423 missing with mode: '1'
Ordinal encoded: range 0-5



In [278]:
# Feature 11: city
# city要做target编码，为防止数据泄露，在划分数据集后再编码
print("="*70)
print("Processing Feature 11: city")
print("="*70)

print(f"Missing values: {df_clean['city'].isna().sum()}")
print(f"Unique cities: {df_clean['city'].nunique()}")

print("Encoder city after split datasets")

# # Import TargetEncoder
# from sklearn.preprocessing import TargetEncoder

# # Target encoding for high cardinality categorical feature
# # Using cross-validation to prevent overfitting
# target_encoder = TargetEncoder(cv=5, smooth='auto', target_type='binary')
# df_clean['city_encoded'] = target_encoder.fit_transform(
#     df_clean[['city']], 
#     df_clean['target']
# )

# # Drop original city column
# df_clean = df_clean.drop('city', axis=1)

# print(f"\nFeature 11 processed: city")
# print(f"   Target encoded with CV=5")
# print(f"   Output: 1 numeric column (city_encoded)")
# print(f"   Range: {df_clean['city_encoded'].min():.4f} to {df_clean['city_encoded'].max():.4f}")
# print(f"   Shape: {df_clean.shape}\n")

Processing Feature 11: city
Missing values: 0
Unique cities: 123
Encoder city after split datasets
Unique cities: 123
Encoder city after split datasets


In [279]:
# Features 12-13: city_development_index & training_hours
print("="*70)
print("Processing Features 12-13: Numeric Features")
print("="*70)

print("city_development_index: Already numeric, no action needed")
print(f"Missing values: {df_clean['city_development_index'].isna().sum()}")
print(f"Range: {df_clean['city_development_index'].min():.3f} to {df_clean['city_development_index'].max():.3f}")

print("\ntraining_hours: Already numeric, no action needed")
print(f"Missing values: {df_clean['training_hours'].isna().sum()}")
print(f"Range: {df_clean['training_hours'].min()} to {df_clean['training_hours'].max()} hours")

print(f"   Shape: {df_clean.shape}\n")


Processing Features 12-13: Numeric Features
city_development_index: Already numeric, no action needed
Missing values: 0
Range: 0.448 to 0.949

training_hours: Already numeric, no action needed
Missing values: 0
Range: 1 to 336 hours
   Shape: (19158, 30)



In [280]:
# Verification: Check cleaned data quality
print("="*80)
print("DATA CLEANING VERIFICATION")
print("="*80)

# 1. Shape comparison
print("\n1. Dataset Shape:")
print(f"   Original: {df.shape}")
print(f"   Cleaned:  {df_clean.shape}")
print(f"   → Rows preserved: {df_clean.shape[0]} (100%)")
print(f"   → Columns changed: {df.shape[1]} → {df_clean.shape[1]} (due to One-Hot encoding)")

# 2. Missing values check
print("\n2. Missing Values:")
missing_after = df_clean.isnull().sum().sum()
if missing_after == 0:
    print(f"No missing values! All {df_clean.shape[0] * df_clean.shape[1]:,} cells are filled")
else:
    print(f"WARNING: {missing_after} missing values found!")
    print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])

# 3. Data types check
print("\n3. Data Types:")
dtypes_summary = df_clean.dtypes.value_counts()
print(f"   {dtypes_summary.to_dict()}")
non_numeric = df_clean.select_dtypes(include=['object']).columns.tolist()
if len(non_numeric) == 0:
    print(f"All features are numeric (ready for modeling)")
else:
    print(f"Non-numeric columns found: {non_numeric}")

# 4. Feature list
print("\n4. Final Feature List:")
feature_cols = [col for col in df_clean.columns if col != 'target']
print(f"   Total features: {len(feature_cols)}")
print(f"   Features: {', '.join(sorted(feature_cols)[:10])}...")

# 5. Display sample
print("\n5. Sample of Cleaned Data (first 3 rows):")
print("="*80)
display(df_clean.head(3))

print("\n" + "="*80)
print("DATA CLEANING COMPLETED SUCCESSFULLY!")
print("="*80)

DATA CLEANING VERIFICATION

1. Dataset Shape:
   Original: (19158, 14)
   Cleaned:  (19158, 30)
   → Rows preserved: 19158 (100%)
   → Columns changed: 14 → 30 (due to One-Hot encoding)

2. Missing Values:
No missing values! All 574,740 cells are filled

3. Data Types:
   {dtype('bool'): 21, dtype('int64'): 6, dtype('float64'): 2, dtype('O'): 1}
Non-numeric columns found: ['city']

4. Final Feature List:
   Total features: 29
   Features: city, city_development_index, company_size, company_type_Early Stage Startup, company_type_Funded Startup, company_type_NGO, company_type_Other, company_type_Public Sector, company_type_Pvt Ltd, company_type_Unknown...

5. Sample of Cleaned Data (first 3 rows):


,city,city_development_index,relevent_experience,education_level,experience,company_size,last_new_job,training_hours,target,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_Unknown,gender_Female,gender_Male,gender_Other,gender_Unknown,major_Arts,major_Business Degree,major_Humanities,major_No Major,major_Other,major_STEM,major_Unknown,enrolled_Full time course,enrolled_Part time course,enrolled_no_enrollment
0,city_103,0.920,1,3,25,-1,1,36,1.0,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,True
1,city_40,0.776,0,3,15,2,5,47,0.0,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True
2,city_21,0.624,0,3,5,-1,0,83,0.0,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,True,False,False



DATA CLEANING COMPLETED SUCCESSFULLY!


## 4. Datasets Split & Target Encoding



### 4.1 Ready for Split & Encoding

In [281]:
from sklearn.model_selection import train_test_split
import numpy as np

# 分离特征和目标变量
X = df_clean.drop('target', axis=1)
y = df_clean['target'].astype(int)

print("="*80)
print("数据集准备")
print("="*80)
print(f"特征矩阵 X: {X.shape}")
print(f"目标变量 y: {y.shape}")
print(f"\n目标变量分布:")
print(f"  Class 0 (不换工作): {(y==0).sum():,} ({(y==0).sum()/len(y)*100:.2f}%)")
print(f"  Class 1 (换工作):   {(y==1).sum():,} ({(y==1).sum()/len(y)*100:.2f}%)")
print(f"  类别比例: {(y==0).sum()/(y==1).sum():.2f}:1")

数据集准备
特征矩阵 X: (19158, 29)
目标变量 y: (19158,)

目标变量分布:
  Class 0 (不换工作): 14,381 (75.07%)
  Class 1 (换工作):   4,777 (24.93%)
  类别比例: 3.01:1


### 4.2 Split

In [282]:
# Step 1: 第一次划分 - 分离出测试集 (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y,
    test_size=0.20,           # 20% 作为测试集
    stratify=y,               # 分层抽样,保持类别比例
    random_state=42           # 固定随机种子,确保可复现
)

print("="*80)
print("第一次划分: 分离测试集")
print("="*80)
print(f"临时集 (Train+Val): {X_temp.shape[0]:,} samples ({X_temp.shape[0]/len(X)*100:.1f}%)")
print(f"测试集 (Test):      {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Step 2: 第二次划分 - 将临时集分为训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.25,           # 0.25 * 80% = 20% (相对于原始数据)
    stratify=y_temp,          # 分层抽样
    random_state=42
)

print("\n" + "="*80)
print("第二次划分: 分离验证集")
print("="*80)
print(f"训练集 (Train):      {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"验证集 (Validation): {X_val.shape[0]:,} samples ({X_val.shape[0]/len(X)*100:.1f}%)")

print("\n" + "="*80)
print("最终数据集划分结果")
print("="*80)
print(f"训练集:   {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%) - 用于训练模型")
print(f"验证集:   {X_val.shape[0]:,} samples ({X_val.shape[0]/len(X)*100:.1f}%) - 用于调参和模型选择")
print(f"测试集:   {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%) - 用于最终评估")
print(f"总计:     {len(X):,} samples (100%)")

第一次划分: 分离测试集
临时集 (Train+Val): 15,326 samples (80.0%)
测试集 (Test):      3,832 samples (20.0%)

第二次划分: 分离验证集
训练集 (Train):      11,494 samples (60.0%)
验证集 (Validation): 3,832 samples (20.0%)

最终数据集划分结果
训练集:   11,494 samples (60.0%) - 用于训练模型
验证集:   3,832 samples (20.0%) - 用于调参和模型选择
测试集:   3,832 samples (20.0%) - 用于最终评估
总计:     19,158 samples (100%)


**Validation quality of splitting**

In [283]:
# 验证每个子集的类别分布
print("="*80)
print("类别分布验证 (分层抽样质量检查)")
print("="*80)

datasets = [
    ("原始数据", y),
    ("训练集", y_train),
    ("验证集", y_val),
    ("测试集", y_test)
]

print(f"\n{'数据集':<15} {'样本数':<12} {'Class 0':<15} {'Class 1':<15} {'比例':<10}")
print("-" * 80)

for name, target in datasets:
    total = len(target)
    class_0 = (target == 0).sum()
    class_1 = (target == 1).sum()
    ratio = class_0 / class_1
    print(f"{name:<15} {total:<12,} {class_0:<7,} ({class_0/total*100:5.2f}%)  {class_1:<7,} ({class_1/total*100:5.2f}%)  {ratio:.2f}:1")

print("\n✓ 如果所有子集的比例接近原始数据,说明分层抽样成功!")

类别分布验证 (分层抽样质量检查)

数据集             样本数          Class 0         Class 1         比例        
--------------------------------------------------------------------------------
原始数据            19,158       14,381  (75.07%)  4,777   (24.93%)  3.01:1
训练集             11,494       8,628   (75.07%)  2,866   (24.93%)  3.01:1
验证集             3,832        2,876   (75.05%)  956     (24.95%)  3.01:1
测试集             3,832        2,877   (75.08%)  955     (24.92%)  3.01:1

✓ 如果所有子集的比例接近原始数据,说明分层抽样成功!


### 4.3 Encoding city with Target Encoding


In [284]:
from sklearn.preprocessing import TargetEncoder

print("="*80)
print("Feature 11: city - Target Encoding (无数据泄露版本)")
print("="*80)

# 初始化Target Encoder
target_encoder = TargetEncoder(
    cv=5,                    # 5折交叉验证,防止过拟合
    smooth='auto',           # 自动平滑,处理低频城市
    target_type='binary'     # 二分类目标
)

# Step 1: 在训练集上fit (学习每个城市的编码)
print("\nStep 1: 在训练集上训练编码器")
print(f"训练集城市数: {X_train['city'].nunique()}")
print(f"样本数: {len(X_train):,}")

target_encoder.fit(X_train[['city']], y_train)
print("✓ 编码器训练完成 (仅使用训练集数据)")

# Step 2: Transform训练集
print("\nStep 2: 编码训练集")
X_train_encoded = X_train.copy()
X_train_encoded['city_encoded'] = target_encoder.transform(X_train[['city']])
X_train_encoded = X_train_encoded.drop('city', axis=1)
print(f"✓ 训练集编码完成")
print(f"   编码范围: {X_train_encoded['city_encoded'].min():.4f} to {X_train_encoded['city_encoded'].max():.4f}")

# Step 3: Transform验证集
print("\nStep 3: 编码验证集 (使用训练集学到的编码)")
X_val_encoded = X_val.copy()
X_val_encoded['city_encoded'] = target_encoder.transform(X_val[['city']])
X_val_encoded = X_val_encoded.drop('city', axis=1)
print(f"✓ 验证集编码完成")
print(f"   验证集城市数: {X_val['city'].nunique()}")

# Step 4: Transform测试集
print("\nStep 4: 编码测试集 (使用训练集学到的编码)")
X_test_encoded = X_test.copy()
X_test_encoded['city_encoded'] = target_encoder.transform(X_test[['city']])
X_test_encoded = X_test_encoded.drop('city', axis=1)
print(f"✓ 测试集编码完成")
print(f"   测试集城市数: {X_test['city'].nunique()}")

print("\n" + "="*80)
print("Target Encoding 完成 (无数据泄露)")
print("="*80)
print(f"训练集: {X_train_encoded.shape}")
print(f"验证集: {X_val_encoded.shape}")
print(f"测试集: {X_test_encoded.shape}")
print(f"\n所有特征现在都是数值型,可以直接用于模型训练!")

Feature 11: city - Target Encoding (无数据泄露版本)

Step 1: 在训练集上训练编码器
训练集城市数: 121
样本数: 11,494
✓ 编码器训练完成 (仅使用训练集数据)

Step 2: 编码训练集
✓ 训练集编码完成
   编码范围: 0.0000 to 1.0000

Step 3: 编码验证集 (使用训练集学到的编码)
✓ 验证集编码完成
   验证集城市数: 112

Step 4: 编码测试集 (使用训练集学到的编码)
✓ 测试集编码完成
   测试集城市数: 117

Target Encoding 完成 (无数据泄露)
训练集: (11494, 29)
验证集: (3832, 29)
测试集: (3832, 29)

所有特征现在都是数值型,可以直接用于模型训练!


### 4.4 Final Check

In [285]:
# 最终数据集摘要
print("="*80)
print("最终数据集摘要")
print("="*80)

print("\n1. 数据集形状:")
print(f"   训练集: X_train_encoded {X_train_encoded.shape}, y_train {y_train.shape}")
print(f"   验证集: X_val_encoded   {X_val_encoded.shape}, y_val   {y_val.shape}")
print(f"   测试集: X_test_encoded  {X_test_encoded.shape}, y_test  {y_test.shape}")

print("\n2. 特征数量:")
print(f"   总特征数: {X_train_encoded.shape[1]}")

print("\n3. 缺失值检查:")
print(f"   训练集缺失: {X_train_encoded.isnull().sum().sum()}")
print(f"   验证集缺失: {X_val_encoded.isnull().sum().sum()}")
print(f"   测试集缺失: {X_test_encoded.isnull().sum().sum()}")

print("\n4. 特征列表 (前15个):")
feature_cols = X_train_encoded.columns.tolist()
for i, col in enumerate(feature_cols[:], 1):
    print(f"   {i:2d}. {col}")


print("\n5. 数据样本 (训练集前3行):")
display(X_train_encoded.head(3))

print("\n" + "="*80)
print("✓ 数据集划分和编码完成! 准备进行模型训练")
print("="*80)

最终数据集摘要

1. 数据集形状:
   训练集: X_train_encoded (11494, 29), y_train (11494,)
   验证集: X_val_encoded   (3832, 29), y_val   (3832,)
   测试集: X_test_encoded  (3832, 29), y_test  (3832,)

2. 特征数量:
   总特征数: 29

3. 缺失值检查:
   训练集缺失: 0
   验证集缺失: 0
   测试集缺失: 0

4. 特征列表 (前15个):
    1. city_development_index
    2. relevent_experience
    3. education_level
    4. experience
    5. company_size
    6. last_new_job
    7. training_hours
    8. company_type_Early Stage Startup
    9. company_type_Funded Startup
   10. company_type_NGO
   11. company_type_Other
   12. company_type_Public Sector
   13. company_type_Pvt Ltd
   14. company_type_Unknown
   15. gender_Female
   16. gender_Male
   17. gender_Other
   18. gender_Unknown
   19. major_Arts
   20. major_Business Degree
   21. major_Humanities
   22. major_No Major
   23. major_Other
   24. major_STEM
   25. major_Unknown
   26. enrolled_Full time course
   27. enrolled_Part time course
   28. enrolled_no_enrollment
   29. city_encoded

5. 数据样本 (训练集

,city_development_index,relevent_experience,education_level,experience,company_size,last_new_job,training_hours,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_Unknown,gender_Female,gender_Male,gender_Other,gender_Unknown,major_Arts,major_Business Degree,major_Humanities,major_No Major,major_Other,major_STEM,major_Unknown,enrolled_Full time course,enrolled_Part time course,enrolled_no_enrollment,city_encoded
2717,0.903,0,2,1,-1,0,122,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,True,False,False,0.178999
17182,0.624,1,3,3,-1,1,270,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,True,False,True,False,False,0.596042
7759,0.804,1,3,14,2,2,48,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,0.198041



✓ 数据集划分和编码完成! 准备进行模型训练
